## Nama : Muhammad Rifqi Abiyyu Ariq
## NIM    : 09011381823114
## Kelas : SK5 Kampus Palembang

# Predicting Parkinson's

Tujuan dari proyek ini adalah untuk menerapkan PCA ke [Parkinson's Disease Classification dataset](http://archive.ics.uci.edu/ml/datasets/Parkinson%27s+Disease+Classification)\[2\] dan kemudian melakukan klasifikasi menggunakan algoritma k-nearest neighbors.

#### Kesimpulan

Mengklasifikasikan data mentah dengan KNN sebelum menerapkan PCA menghasilkan CCR 0,65789 saat melatih model setelah mengurangi data PCA menghasilkan CCR 0,67763, yang sedikit lebih tinggi. Akurasi yang lebih tinggi saat menggunakan data yang berkurang mungkin disebabkan oleh PCA mengurangi kebisingan dan outlying feataures yang tidak mempengaruhi prediksi yang benar, sehingga dengan menghapusnya model membaik. 

### 1. Persiapan dan Pra-proses
Muat data 'pd_speech_X.csv' dan label terkait 'pd_speech_Y.csv'. Normalkan data dengan mengurangi rata-rata dan membagi dengan varians untuk setiap atribut/fitur. Bagi data dan label menjadi kumpulan pelatihan dan pengujian menggunakan train/test

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

# load data
X_df = pd.read_csv("pd_speech_X.csv")
Y = pd.read_csv("pd_speech_Y.csv")
X_df.head()

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36
0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,...,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405
1,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,...,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780
2,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,...,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666
3,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,...,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603
4,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,0.000044,...,7.8832,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164


In [ ]:
X_norm = X_df - np.mean(X_df, axis=0)
X_norm /= np.var(X_norm, axis=0)
X_norm.head()

In [16]:
#split data into test and train
X_train, X_test, Y_train, Y_test = train_test_split(X_norm, Y, test_size=0.2)
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)

(604, 753)
(604, 1)
(152, 753)


### 2. KNN dengan data mentah
Dengan menggunakan data train dan label, train classifier k-nearest neighbors dengan k=3. Laporkan tingkat klasifikasi (CRR) yang benar pada data pengujian.

In [17]:
# apply knn
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, Y_train)

# test the model
y_pred_test = neigh.predict(X_test)

ccr_test = (y_pred_test == Y_test['class']).mean()
print('Test ccr = {:.5f}'.format(ccr_test))

Test ccr = 0.65789


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


### 3. KNN setelah PCA
Terapkan PCA (gunakan jumlah komponen = 5) ke data train. Selanjutnya, ubah data train menjadi representasinya di ruang dimensi bawah yang dipelajari oleh model PCA. Dengan menggunakan representasi ini, train classifier k-nearest neighbors dengan k=3. Laporkan tingkat klasifikasi yang benar pada data pengujian.

In [18]:
# pca using 5 components
pca = PCA(n_components=5)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [19]:
# apply knn on transformed data
neigh_pca = KNeighborsClassifier(n_neighbors=3)
neigh_pca.fit(pca.transform(X_train), Y_train['class'])

# test the model
y_pred_test = neigh_pca.predict(pca.transform(X_test))
ccr_test = (y_pred_test == Y_test['class']).mean()
print('Test ccr = {:.5f}'.format(ccr_test))

Test ccr = 0.67763


## Kutipan
1. Sakar, C.O., Serbes, G., Gunduz, A., Tunc, H.C., Nizam, H., Sakar, B.E., Tutuncu, M., Aydin, T., Isenkul, M.E. and Apaydin, H., 2018. A comparative analysis of speech signal processing algorithms for Parkinsonâ€™s disease classification and the use of the tunable Q-factor wavelet transform. Applied Soft Computing, DOI